In [30]:
from data_storage import create_connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


In [31]:
connection = create_connection("../database/crypto_billionairs.db")
scaler = MinMaxScaler()

In [32]:
def apply_random_forest(X_train, X_test, y_train, y_test, table, db_connection):
    
    clf = RandomForestClassifier(criterion="entropy", min_samples_split= 0.01, min_samples_leaf= 0.005, max_depth=10, class_weight="balanced_subsample")
    clf.fit(X_train, y_train)
    #print(X_test.head())
    X_test_normalized = X_test.copy()
    X_test_normalized[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_test_normalized[["open", "close", "high", "low", "volume"]])
    y_pred = clf.predict(X_test_normalized)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_random_forest", db_connection, if_exists="replace")
    return "random forest", f1score, f1score_macro, recall, precision

In [33]:
def apply_knn(X_train, X_test, y_train, y_test, table, db_connection):
    
    neigh = KNeighborsClassifier(weights="uniform", n_neighbors=3, algorithm="ball_tree")
    neigh.fit(X_train, y_train)
    
    X_test_normalized = X_test.copy()
    X_test_normalized[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_test_normalized[["open", "close", "high", "low", "volume"]])
    y_pred = neigh.predict(X_test_normalized)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_knn", db_connection, if_exists="replace")
    
    return "k-nearest neighbour", f1score, f1score_macro, recall, precision

In [34]:
def apply_support_vector_machine(X_train, X_test, y_train, y_test, table, db_connection):
    
    svc = SVC(kernel="poly", degree=4, C=1)
    svc.fit(X_train, y_train)
    
    X_test_normalized = X_test.copy()
    X_test_normalized[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_test_normalized[["open", "close", "high", "low", "volume"]])
    y_pred = svc.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_svc", db_connection, if_exists="replace")
    
    return "support vector classifier", f1score, f1score_macro, precision, recall

In [35]:
def apply_mlp(X_train, X_test, y_train, y_test, table, db_connection):
    
    mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation="tanh", solver="lbfgs", learning_rate="constant", learning_rate_init=2e-5, tol=1e-5)
    mlp.fit(X_train, y_train)
    
    X_test_normalized = X_test.copy()
    X_test_normalized[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_test_normalized[["open", "close", "high", "low", "volume"]])
    y_pred = mlp.predict(X_test_normalized)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_mlp", db_connection, if_exists="replace")
    
    return "mlp classifier", f1score, f1score_macro, precision, recall 

In [36]:
def apply_logistic_regression(X_train, X_test, y_train, y_test, table, db_connection):
    
    lg = LogisticRegression(solver="liblinear", penalty="l1", C=1)
    lg.fit(X_train, y_train)
    
    X_test_normalized = X_test.copy()
    X_test_normalized[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_test_normalized[["open", "close", "high", "low", "volume"]])
    
    y_pred = lg.predict(X_test_normalized)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_logistic_regression", db_connection, if_exists="replace")
    
    return "logistic regression", f1score, f1score_macro, recall, precision
    

In [37]:
def apply_ml_algorithms(db_connection):
    
    df_ml = pd.DataFrame(columns = range(6))
    df_ml.columns = ["table_name", "model", "f1-score weighted", "f1-score macro", "recall macro", "precision macro"]
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and not 'equity_curve' in name and '_pooling' not in name]
    ros = RandomOverSampler(random_state=0)
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
    
        y = df_temp["buy_indicator"] + df_temp["short_indicator"]
        y = y.fillna(0)
        y = y.astype(str)
        
        X = df_temp.drop(["return", "buy_indicator", "short_indicator", "close_buy_indicator", "close_short_indicator", "time", "index"], axis=1)
        
        X_train = X.iloc[:-365]
        print(table)
        X_train[["open", "close", "high", "low", "volume"]] = scaler.fit_transform(X_train[["open", "close", "high", "low", "volume"]])
        
        X_test = X.iloc[-365:]
        
        y_train = y.iloc[:-365]
        y_test = y.iloc[-365:]
        
        X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
        
        string, score, f1score_macro, recall, precision  = apply_random_forest(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_knn(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_support_vector_machine(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_mlp(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_logistic_regression(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
    return df_ml
        

In [40]:
%%capture
df_ml = apply_ml_algorithms(connection)

In [39]:
df_ml

,table_name,model,f1-score weighted,f1-score macro,recall macro,precision macro
0,ADA_1min_complete_1day_preprocessed_1day_features,random forest,0.509773,0.330682,0.341452,0.354525
1,ADA_1min_complete_1day_preprocessed_1day_features,k-nearest neighbour,0.165795,0.143807,0.361595,0.292505
2,ADA_1min_complete_1day_preprocessed_1day_features,support vector classifier,0.530515,0.368213,0.387322,0.391644
3,ADA_1min_complete_1day_preprocessed_1day_features,mlp classifier,0.038134,0.085919,0.049315,0.333333
4,ADA_1min_complete_1day_preprocessed_1day_features,logistic regression,0.619459,0.361168,0.362699,0.398165
...,...,...,...,...,...,...
135,TRX_1min_complete_1day_preprocessed_1day_features,random forest,0.567557,0.371240,0.422273,0.389065
136,TRX_1min_complete_1day_preprocessed_1day_features,k-nearest neighbour,0.680972,0.291731,0.333333,0.259361
137,TRX_1min_complete_1day_preprocessed_1day_features,support vector classifier,0.019669,0.061360,0.033976,0.316239
138,TRX_1min_complete_1day_preprocessed_1day_features,mlp classifier,0.079406,0.102546,0.295960,0.290736
